In this competition, we need to extract "the mention of datasets". <br>
However, I don't have a clear answer as to what constitutes a mention at this stage.<br>

Therefore, we will use known labels to dig deeper into the sentences that contain specific words but are not explicitly indicated in the train.csv.

In [ ]:
import pandas as pd
import os
import json
from tqdm import tqdm
import re
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px

In [ ]:
TRAIN = "../input/coleridgeinitiative-show-us-the-data/train"
TEST  = "../input/coleridgeinitiative-show-us-the-data/test"

In [ ]:
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")

In [ ]:
paper_sentense = []
for file in tqdm(os.listdir(TRAIN)):
    
    texts = []
    
    ids = file.split(".")[0]
    file_path = os.path.join(TRAIN, file)
    with open(file_path, "r") as f:
        json_datasets = json.load(f)
    
    for json_dataset in json_datasets:
        for k, v in json_dataset.items():
            if k == "text":
                texts.append(v)
    
    paper_sentense.append([ids, " ".join(texts)])
                

In [ ]:
paper_sentense_df = pd.DataFrame(paper_sentense, columns=["Id", "Sentense"])

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

# Contain AND Mentioned

Percentage of those that are judged to be actually mentioning the sentence among those that contain the sentence.

## Not Cleaned

In [ ]:
labels = train_df["dataset_label"].unique()
metioned_rate = []
for label in tqdm(labels):
    mentioned_ids = train_df[train_df["dataset_label"] == label]["Id"].unique().tolist()
    paper_sentense_df[f"is_{label}_contains"] = paper_sentense_df["Sentense"].str.contains(label)
    mean = paper_sentense_df[paper_sentense_df[f"is_{label}_contains"]]["Id"].isin(mentioned_ids).mean()
    metioned_rate.append([label, mean])
    
metioned_rate_df = pd.DataFrame(metioned_rate, columns=["Label", "MentionedRate"]).sort_values("MentionedRate")

In [ ]:
px.bar(data_frame=metioned_rate_df,y=metioned_rate_df["Label"],x="MentionedRate", orientation = 'h', width=1200, height=3000)

## cleaned

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

In [ ]:
train_df["cleaned_pre_label"] = train_df["cleaned_label"].apply(lambda x: x[:-1] if x[-1] == " " else x)
cleaned_labels = train_df[["cleaned_label", "cleaned_pre_label"]].drop_duplicates()
cleaned_metioned_rate = []
cleaned_paper_sentense_df = paper_sentense_df[["Id", "Sentense"]]
cleaned_paper_sentense_df["Sentense"] = cleaned_paper_sentense_df["Sentense"].progress_apply(totally_clean_text) 
for (label, pre_label) in tqdm(cleaned_labels.values):
    mentioned_ids = train_df[train_df["cleaned_label"] == label]["Id"].unique().tolist()
    cleaned_paper_sentense_df[f"is_{label}_contains"] = cleaned_paper_sentense_df["Sentense"].str.contains(pre_label)
    mean = cleaned_paper_sentense_df[cleaned_paper_sentense_df[f"is_{label}_contains"]]["Id"].isin(mentioned_ids).mean()
    cleaned_metioned_rate.append([label, mean])
    
cleaned_metioned_rate_df = pd.DataFrame(cleaned_metioned_rate, columns=["Label", "MentionedRate"]).sort_values("MentionedRate")

In [ ]:
px.bar(data_frame=cleaned_metioned_rate_df,y=cleaned_metioned_rate_df["Label"],x="MentionedRate", orientation = 'h', width=1200, height=3000)

Regarding the known labels, most of them are included = mention, but some of them seem to be less so.<br>

Some obvious dataset names(Ex: aging intergrated database agid) were found that were not "mention".

In [ ]:
train_df

# Mentioned "National Education Longitudinal Study"

In [ ]:
q1 = paper_sentense_df[paper_sentense_df["Sentense"].str.contains("National Education Longitudinal Study")]
a1 = train_df[train_df["dataset_title"].str.contains("National Education Longitudinal Study")]["Id"].unique()
q1["is_answer"] = q1["Id"].isin(a1.tolist()) * 1

## No Mentioned

13 IDs contain "National Education Longitudinal Study", but train.csv mentioned it.

b2ad2a12-78ce-4c65-ba18-e5b7a82cf449
> Until recently, longitudinal analyses were limited to use of the **National Education Longitudinal Study** 1988 (NELS:88) data -a set that does not account for the "second wave" of accountability measures (Dee & Jacob, 2006)

Not present in references

b6f4c581-74cd-4b58-8006-b891cffad287	
> Thus, this chapter provides a background for the two analyses on S&E program entrance and completion that are described in chapters 3 and 4. Chapter 2 describes the study approach used in the two analyses. The first analysis, in chapter 3, uses data from the **National Education Longitudinal Study** of 1988 (NELS:88)

Present in references

26f6d309-d11a-4335-ac48-27a33c9d6953
> However, in contrast to these findings, cross-sectional studies of large national datasets tend to report a negative relationship between parental homework help and student achievement, including studies using data from the **National Education Longitudinal Study** (NELS:88) (e.g., Fan 2001;Fan and Chen 2001;McNeal 1999;Muller 1995;Peng and Wright 1994;Sui-Chu and Willms 1996) and other more recent analyses of national and state-level datasets (Moroni et al. 2015;Robinson and Harris 2014)

Not present in references

## Mentioned

f8b03c87-9d1a-4f20-b76b-cb6c69d447b2
> For this reason, these data are frequently provided to organization clients in repackaged form without attribution as to origin, and the extent of reliance on federal expertise and data collection capabilities is thus obscured. 6 Three longitudinal studies have been conducted to date: the National Longitudinal Study of the High School Graduating Class of 1972 (NLS-72), starting with a sample of 12th graders; the High School and Beyond Study (IISB), starting with two samples, one of the sophomore (grade 10) and a second of the senior (grade 12) class of 1980; and the **National Education Longitudinal Study** of 1988 (NELS-88), starting with a sample of students in grade 8 as of 1988

Not present in references <br>
Not in the text, but in the remarks

f8c8eb2a-4e31-4661-9fed-6ab4f3280ba4
> Both surveys include a variety of items that have been used in other studies such as the General Social Survey and the **National Education Longitudinal Study** of 1988-92 that ask about religious participation and school experiences. Survey items measuring parenting style are based on similar items in the ECLS-Kindergarten Spring Parent Interview Questionnaire (http://nces.ed.gov/ecls/ kindergarten/questionnaires.asp) and the Child\'s Report of Parent Behavior Inventory (Shaefer, 1965) . Questions about basic demographic characteristics of the respondents and family activities were also included in the surveys.\nThe ESM, created by Csikszentmihalyi and colleagues (Csikszentmihalyi & Larson, 1987) , is a form of time diary that examines the daily lives of individuals, including what they are doing and how they feel

Not present in references

167f0d62-9912-4279-be98-5fcac4836889
> Using data from the **National Education Longitudinal Study** of 1988 (NELS) and the National Postsecondary Student Aid Study (NPSAS), this study examines access to postsecondary education by individuals in different income and test score groups. While many studies have found a statistically significant effect of income on college enrollment,\' less attention has been paid to the effect of family income after controlling for student achievement. 

Not present in references(Maybe)

**Still not clear on the differences.**

# Include "ADNI" sentense and Include "ADNI" word

In [ ]:
adnis = train_df[train_df["cleaned_label"] == "adni"]["Id"].unique()

In [ ]:
paper_sentense_df["CleanedSentense"] = paper_sentense_df["Sentense"].progress_apply(totally_clean_text) 

paper_sentense_df["is_in_sentense_adni"] = paper_sentense_df["CleanedSentense"].str.contains("adni") * 1
paper_sentense_df["is_in_word_adni"] = paper_sentense_df["CleanedSentense"].apply(lambda x: "adni" in x.split()) * 1
paper_sentense_df["is_adni"] = paper_sentense_df["Id"].isin(adnis) * 1

paper_sentense_df["is_relate_adni"] = paper_sentense_df[["is_in_sentense_adni","is_in_word_adni","is_adni"]].sum(axis=1)

In [ ]:
paper_sentense_df["is_relate_adni"].value_counts()

## existing as sentense, not existing as word, not label(1, 0, 0)

ID: 221312c6-31ad-406d-9da6-6e416a07f74b
> m**adni**ck

## existing as sentense, not existing as word, label(1, 0, 1)

ID: 63031adf-e584-481b-8b78-957b17c513bf
> Drapves5gWxySRtDrYbx5bv£Gr$**adnI**1!

?????????????????????

## existing as sentense,  existing as word, not label(1, 1, 0)

ID: 91a1f72f-2884-48d9-a04d-9baae3e234b8
> For example, an examination of **ADNI** participants over age 65 suggested that performance on Trails B would vary within age-specific subgroups in this cohort (see Fig. 7 showing mean and standard error for Trails B in four older age subgroups).